In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/응용통계학과 공모전')

# 요약
1. 채용공고를 문장단위로 분리  
B) 직무 기준이 아닌 하는일을 기준으로 텍스트 군집화하여 인사이트를 도출하기 위함
2. 문장별 명사 추출
3. SBERT를 통해 문장 텍스트 임베딩
4. 문장별 vector를 통한 K-means clustering
5. kmeans_label을 문장별 명사에 할당
6. 채용공고의 kmeans_label이 할당된 문장별 비율을 계산하여 사용자 input과 유사한 채용공고를 추천해주기 위한 데이터베이스 구축
7. 문장 단위, 명사 단위 키워드 추출 및 시각화를 위한 데이터베이스 구축

In [ ]:
!pip install transformers
!pip install soynlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 5.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

# 데이터 불러오기
BERTopic은 nouns 단위, keyBERT는 sentences 단위로 토픽모델링을 진행.

따라서 두가지 nouns와 sentences 데이터프레임을 생성.

In [ ]:
df = pd.read_csv('./data/wanted_final_with_token.csv')
df = df.reset_index()
df.head(3)

,index,공고명,직무,유사 직무,업종,회사명,주요업무,자격요건,우대사항,nouns,sentence
0,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"['언어', '성능', '고도화', 'LLM', '학습', 'finetuning',...",['기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니...
1,1,AI 컴파일러 개발자 (1년 이상),머신러닝 엔지니어,머신러닝 엔지니어,"IT, 컨텐츠",티에스엔랩,MLIR 기반의 AI 컴파일러 관련 프로젝트에 참여합니다,웬만한 중견기업 못지 않은 복지 정책을 지원 임직원 연차 구분 없이 20일 휴가 ...,NaN,"['ML', 'AI', '컴파일러', '프로젝트', '참여', '정직', '원', ...","['MLIR 기반의 AI 컴파일러 관련 프로젝트에 참여합니다', '웬만한 중견기업 ..."
2,2,LLM 연구원 (자연어 처리),머신러닝 엔지니어,머신러닝 엔지니어,"IT, 컨텐츠",딥노이드(Deepnoid),LLM 모델 연구 및 고도화LLM 연구의 서비스화를 위한 개발 지원,"경력 3년 이상 또는 관련 학과 석사 학위 소지자PyTorch, Tensorflo...","인공지능 관련 대회해커톤, 캐글, 그랜드챌린지 등의 출전 경험 및 입상Multi...","['LLM', '모델', '연구', '고도화', 'LLM', '연구', '학과', ...","['LLM 모델 연구 및 고도화', 'LLM 연구의 서비스화를 위한 개발 지원', ..."


In [ ]:
len(df["index"].value_counts())

1145

In [ ]:
df['sentence'][0]

"['기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다', 'LLM 학습 및 finetuning을 통해 내부적으로 필요한 LLM 개발을 진행합니다', 'AI ,  ML 개발 방향성에 대한 전략을 수립합니다', '기업 고객에게 필요한 LLM 학습 데이터 구축 가이드라인 수립 및 사내 데이터 구축을 지원합니다', 'AI ,  ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분', 'ML 프레임워크PyTorch,  TensorFlow 등를 자신있게 다룰 수 있는 분', '모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이 있는 분', '자신이 만든 모델을 실제 상용 서비스에 적용하고 운영한 경험이 있는 분', 'AWS나 Azure 등 클라우드 환경을 이용해 개발한 경험이 있는 분', 'Github에 공개 가능한 ML 프로젝트가 있는 분']"

In [ ]:
df['nouns'][0]

"['언어', '성능', '고도화', 'LLM', '학습', 'finetuning', 'LLM', 'AI', 'ML', '방향', '고객', 'LLM', '학습', '데이터', '구축', '가이드', '데이터', '구축', 'ML', '등', '박사', '학위', 'ML', '프레임워크', 'PyTorch', '자신', '분모델', '공개', '모델', 'finetuning', '성능', '향상', '자신', '모델', '운영', 'AWS', '등', '클라우드', '이용', 'Github', '공개', 'ML', '프로젝트', '가']"

In [ ]:
import ast
df['sentence'] = df['sentence'].apply(ast.literal_eval)
df['nouns'] = df['nouns'].apply(ast.literal_eval)

In [ ]:
df['sentence'][0]

['기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다',
 'LLM 학습 및 finetuning을 통해 내부적으로 필요한 LLM 개발을 진행합니다',
 'AI ,  ML 개발 방향성에 대한 전략을 수립합니다',
 '기업 고객에게 필요한 LLM 학습 데이터 구축 가이드라인 수립 및 사내 데이터 구축을 지원합니다',
 'AI ,  ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분',
 'ML 프레임워크PyTorch,  TensorFlow 등를 자신있게 다룰 수 있는 분',
 '모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이 있는 분',
 '자신이 만든 모델을 실제 상용 서비스에 적용하고 운영한 경험이 있는 분',
 'AWS나 Azure 등 클라우드 환경을 이용해 개발한 경험이 있는 분',
 'Github에 공개 가능한 ML 프로젝트가 있는 분']

In [ ]:
df['nouns'][0]

['언어',
 '성능',
 '고도화',
 'LLM',
 '학습',
 'finetuning',
 'LLM',
 'AI',
 'ML',
 '방향',
 '고객',
 'LLM',
 '학습',
 '데이터',
 '구축',
 '가이드',
 '데이터',
 '구축',
 'ML',
 '등',
 '박사',
 '학위',
 'ML',
 '프레임워크',
 'PyTorch',
 '자신',
 '분모델',
 '공개',
 '모델',
 'finetuning',
 '성능',
 '향상',
 '자신',
 '모델',
 '운영',
 'AWS',
 '등',
 '클라우드',
 '이용',
 'Github',
 '공개',
 'ML',
 '프로젝트',
 '가']

# 문장분리 (df_sentence)

문장분리하여 채용공고를 각 문장으로 분리.  
분리된 문장으로 SBERT를 통해 텍스트 임베딩하여 문장별 벡터를 산출하고 텍스트 군집화 진행.  
향후 WordCloud, BERTopic, keyBERT 등의 키워드추출 및 시각화 알고리즘에 사용.

In [ ]:
# sentence칼럼 문장분리
df_sentence = df.explode('sentence')
df_sentence = df_sentence.reset_index(drop=True)
df_sentence['sentence'] = df_sentence['sentence'].str.strip()
df_sentence.head(3)

,index,공고명,직무,유사 직무,업종,회사명,주요업무,자격요건,우대사항,nouns,sentence
0,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"[언어, 성능, 고도화, LLM, 학습, finetuning, LLM, AI, ML...",기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다
1,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"[언어, 성능, 고도화, LLM, 학습, finetuning, LLM, AI, ML...",LLM 학습 및 finetuning을 통해 내부적으로 필요한 LLM 개발을 진행합니다
2,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"[언어, 성능, 고도화, LLM, 학습, finetuning, LLM, AI, ML...","AI , ML 개발 방향성에 대한 전략을 수립합니다"


In [ ]:
df_sentence["sentence"]

0            기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다
1          LLM 학습 및 finetuning을 통해 내부적으로 필요한 LLM 개발을 진행합니다
2                            AI ,  ML 개발 방향성에 대한 전략을 수립합니다
3        기업 고객에게 필요한 LLM 학습 데이터 구축 가이드라인 수립 및 사내 데이터 구축...
4                  AI ,  ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분
                               ...                        
17393    데이터베이스 개발 경험 TSQL,  Ansi SQL,  Shell Script,  ...
17394             데이터 모델링에 대한 경험모델링 절차 ,  정규화 및 비정규화 있으신 분
17395           Windows,  Linux,  Cloud 환경에 대한 기본 지식 있으신 분
17396                             데이터 모델 검수 및 가이드 경험 있으신 분
17397                         데이터 흐름 분석 및 데이터 표준화 경험 있으신 분
Name: sentence, Length: 17398, dtype: object

# soynlp 형태소 분석, 명사 추출
BERTopic, keyBERT에 쓰일 명사 단위 분리.  
군집별로 명사 단위 토픽 모델링을 진행할 예정이므로 soynlp를 통해 문장별 형태소 분석 및 명사 추출.  

In [ ]:
#불용어 처리

#불용어 사전 불러오기
with open('/content/drive/MyDrive/응용통계학과 공모전/data/korean_stopwords_응통.txt','r',encoding='utf-8-sig') as f:
    stopwords_list=[]
    example =f.readlines()
    for line in example:
        stopwords_list.append(line.strip())

#추가하고 싶은 불용어가 있다면 다음과 같이 넣어서 사용
add = ['근무조건','고용형태','근무장소','급여조건','합류','서류','전형','인터뷰','합격','유의사항','공고','모집','채용','정규직','면접','전형','근무시','출근']

#불용어 사전 정의
stop = add+stopwords_list

In [ ]:
# WordExtractor의 parameter
# def __init__(max_left_length=10,
#            max_right_length=6,
#            min_frequency=5,
#            verbose_points=100000,
#            min_cohesion_forward=0.1,
#            min_cohesion_backward=0.0,
#            max_droprate_cohesion=0.95,
#            max_droprate_leftside_frequency=0.95,
#            min_left_branching_entropy=0.0,
#            min_right_branching_entropy=0.0,
#            min_left_accessor_variety=0,
#            min_right_accessor_variety=0,
#            remove_subwords=True)


# 형태소에 해당하는 단어를 분리하는 학습 수행
from soynlp.word import WordExtractor

# 다른 파라미터는 그냥 두고, min_frequency만 1로 설정.
word_extractor = WordExtractor(min_frequency=1)

# 단어 토큰화를 위해 df['combined'] 학습
word_extractor.train(df_sentence['sentence'].astype(str))

# cohesion, branching entropy, accessor variety score 계산
# cohension : 조건부확률을 통해 한글자씩 예측. cohension 값이 높은 위치가 하나의 단어를 이루고 있을 가능성이 큼.
# branching entropy : 확률분포의 엔트로피를 통  해서 계산
# accessor variety : 확률분포 없이, 다음 글자로 등장할 수 있는 경우의 수 계산
word_score = word_extractor.extract()

training was done. used memory 0.254 Gb
all cohesion probabilities was computed. # words = 48643
all branching entropies was computed # words = 40074
all accessor variety was computed # words = 40074


In [ ]:
# 총 단어 후보 개수 확인
len(word_score)

28237

In [ ]:
# soynlp의 토큰화 방식.

# L-토큰화
# 한국어의 경우 공백(띄어쓰기)으로 분리된 하나의 문자열은 ‘L 토큰 + R 토큰; 구조인 경우가 많음
# 왼쪽에 오는 L 토큰은 체언(명사, 대명사)이나 동사, 형용사 등이고 오른쪽에 오는 R 토큰은 조사, 동사, 형용사 등이다.
# 여러가지 길이의 L 토큰의 점수를 비교하여 가장 점수가 높 L단어를 찾는 방법
from soynlp.tokenizer import LTokenizer

# 최대점수토큰화
# 띄어쓰기가 되어 있지 않는 긴 문자열에서 가능한 모든 종류의 부분문자열을 만들어서 가장 점수가 높은 것을 하나의 토큰으로
# 우리는 띄어쓰기가 되어있는 텍스트니까 사용하지 않을 예정
from soynlp.tokenizer import MaxScoreTokenizer

# 규칙기반토큰화
# 우리가 사용하기에 적절치않음.
from soynlp.tokenizer import RegexTokenizer

In [ ]:
from soynlp.noun import LRNounExtractor

# 명사 추출을 위해 df_sentence2['sentence'] 학습
noun_extractor = LRNounExtractor()
nouns = noun_extractor.train_extract(df_sentence['sentence'].astype(str))

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (5648, 3377) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 921 nouns are extracted


In [ ]:
# ex) '통계': NounScore(frequency=137, score=1.0)
# 추출한 명사의 수
len(nouns)

921

In [ ]:
# 기존 konlpy 형태소분석기에서는 외국어로 판단됐으나, noun_extractor 학습 후에는 SQL을 명사라고 판단
noun_extractor.is_noun('SQL')

True

In [ ]:
# 학습 데이터에 존재하지 않는 건 명사라고 인식 못하는 듯
# 토큰화 과정에서 별도의 외부 데이터 혹은 사전이 필요 없는 비지도 학습이라는 장점이 있지만, 그만큼 학습데이터에 민감하게 작용함.
noun_extractor.is_noun('생물')

False

In [ ]:
noun_extractor.is_noun('finetuning')

True

In [ ]:
# 명사추출기의 score와 cohesion score를 함께 이용해서 토큰화 하기

cohesion_score = {word:score.cohesion_forward for word, score in word_score.items()}
noun_scores = {noun:score.score for noun, score in nouns.items()}

combined_scores = {noun:score + cohesion_score.get(noun, 0)
    for noun, score in noun_scores.items()}

combined_scores.update(
    {subword:cohesion for subword, cohesion in cohesion_score.items()
    if not (subword in combined_scores)}
)

LTokenizer = LTokenizer(scores=combined_scores)

In [ ]:
# 토큰화 한뒤, 토큰마다 명사 판별 진행하여 명사 추출하기. nouns 컬럼에 결과 저장.

train_list=list(df_sentence.sentence)
num_result = []
for i in range(len(train_list)):
    num = []
    tok = LTokenizer.tokenize(str(train_list[i]))
    for j in tok:
        if noun_extractor.is_noun(j) and j not in stop:
            num.append(j)
    num_result.append(num)

df_sentence['nouns'] = num_result
df_sentence.head()

,index,공고명,직무,유사 직무,업종,회사명,주요업무,자격요건,우대사항,nouns,sentence
0,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"[언어, 성능, 고도화]",기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다
1,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"[LLM, 학습, finetuning, LLM]",LLM 학습 및 finetuning을 통해 내부적으로 필요한 LLM 개발을 진행합니다
2,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"[AI, ML, 방향]","AI , ML 개발 방향성에 대한 전략을 수립합니다"
3,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"[고객, LLM, 학습, 데이터, 구축, 가이드, 데이터, 구축]",기업 고객에게 필요한 LLM 학습 데이터 구축 가이드라인 수립 및 사내 데이터 구축...
4,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"[AI, ML, 등, 박사, 학위]","AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분"


# BertModel, BertTokenizer, K-means clustering

In [ ]:
import torch
import pickle
from tqdm.notebook import tqdm

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from transformers import AutoTokenizer, AutoModel

In [ ]:
# 문장 데이터
# sentences = [
#     "회사에서는 웹 개발자를 모집합니다.",
#     "데이터 분석가를 찾고 있습니다.",
#     "앱 개발 경험이 있는 개발자를 모집합니다.",
#     "머신러닝 엔지니어를 구합니다.",
#     "프론트엔드 개발자를 채용합니다."
# ]
sentences = df_sentence['sentence'].tolist()

# Hugging Face의 Pretrained Language Model을 사용하여 문장을 벡터화
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 문장을 벡터화하는 함수
def sentence_to_vector(sentence):
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, padding="longest")
    inputs = np.array([input_ids])
    with torch.no_grad():
        outputs = model(torch.tensor(inputs))
    sentence_vector = np.mean(outputs.last_hidden_state.numpy(), axis=1)
    return sentence_vector

# 문장을 벡터화하여 저장할 리스트
vectors = []

# 문장을 벡터화하여 리스트에 저장
for sentence in tqdm(sentences):
    vector = sentence_to_vector(sentence)
    vectors.append(vector)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

  0%|          | 0/15771 [00:00<?, ?it/s]

In [ ]:
vectors = [vector.tolist() for vector in vectors]

In [ ]:
len(vectors[0])

1

In [ ]:
vectors = [vector[0] for vector in vectors]

In [ ]:
len(vectors)

15771

In [ ]:
len(df_sentence)

15771

In [ ]:
df_vectors = pd.DataFrame({"vector": vectors})

In [ ]:
df_vectors.head(3)

,vector
0,"[0.03565007075667381, -0.7653368711471558, 0.3..."
1,"[0.026964180171489716, -0.3026787340641022, 0...."
2,"[-0.10177028179168701, -0.4882239103317261, 0...."


In [ ]:
df_sentence = pd.concat([df_sentence, df_vectors], axis = 1)

In [ ]:
# vector를 기준으로 K-means clustering을 진행하기위해 numpy array 형태로 변경
X = np.array(df_sentence["vector"].tolist())

In [ ]:
# Elbow Method
distortions = []
K = range(2,10)
for k in tqdm(K):
     kmeanModel = KMeans(n_clusters=k)
     kmeanModel.fit(X)
     distortions.append(kmeanModel.inertia_)

plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-', color='violet')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
# Silhouette Score
silhouette_scores = []

K = range(2, 10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(X)
    labels = kmeanModel.labels_

    silhouette_avg = silhouette_score(X, labels)
    silhouette_scores.append(silhouette_avg)

optimal_k = K[silhouette_scores.index(max(silhouette_scores))]


plt.figure(figsize=(16, 8))
plt.plot(K, silhouette_scores, 'bx-', color='violet')
plt.xlabel('k')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score showing the optimal k')
plt.axvline(x=optimal_k, color='red', linestyle='--', label='Optimal k')
plt.legend()
plt.show()

print(f"Optimal k: {optimal_k}")

In [ ]:
# K-means clustering
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=4, random_state=42)

In [ ]:
labels = kmeans.labels_
df_sentence['kmeans_label'] = labels
df_sentence.head(3)

,index,공고명,직무,유사 직무,업종,회사명,주요업무,자격요건,우대사항,nouns,sentence,vector,kmeans_label
0,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"['언어', '성능', '고도화', 'LLM', '학습', 'finetuning',...",기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다,"[0.03565007075667381, -0.7653368711471558, 0.3...",2
1,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"['언어', '성능', '고도화', 'LLM', '학습', 'finetuning',...",LLM 학습 및 finetuning을 통해 내부적으로 필요한 LLM 개발을 진행합니다,"[0.026964180171489716, -0.3026787340641022, 0....",3
2,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"['언어', '성능', '고도화', 'LLM', '학습', 'finetuning',...","AI , ML 개발 방향성에 대한 전략을 수립합니다","[-0.10177028179168701, -0.4882239103317261, 0....",3


In [ ]:
df_sentence["kmeans_label"].value_counts()

3    5428
1    4407
2    3145
0    2791
Name: kmeans_label, dtype: int64

## 채용공고 추천
사용자에게 맞는 채용공고를 추천해주기위해 채용공고별 군집 비율을 계산하는 리스트 칼럼을 생성

In [ ]:
# 인덱스를 기준으로 그룹화하고 각 그룹에 대한 KMeans 라벨 비율을 리스트에 할당
label_ratios = df_sentence.groupby('index')['kmeans_label'].value_counts(normalize=True).unstack(fill_value=0)
label_ratios

kmeans_label,0,1,2,3
index,,,,
0,0.100000,0.100000,0.400000,0.400000
1,0.636364,0.090909,0.090909,0.181818
2,0.166667,0.166667,0.250000,0.416667
3,0.272727,0.272727,0.181818,0.272727
4,0.181818,0.363636,0.181818,0.272727
...,...,...,...,...
1139,0.454545,0.363636,0.181818,0.000000
1141,0.250000,0.125000,0.250000,0.375000
1142,0.083333,0.083333,0.250000,0.583333


In [ ]:
# 인덱스를 기준으로 그룹화하고 각 그룹에 대한 KMeans 라벨 비율을 리스트에 할당
label_ratios = df_sentence.groupby('index')['kmeans_label'].value_counts(normalize=True).unstack(fill_value=0)

df_sentence['label_ratios'] = df_sentence['index'].map(label_ratios.to_dict(orient='index'))
df_sentence['label_ratios'] = df_sentence['label_ratios'].apply(lambda x: list(x.values()))

df_sentence.head(1)

,index,공고명,직무,유사 직무,업종,회사명,주요업무,자격요건,우대사항,nouns,sentence,vector,kmeans_label,label_ratios
0,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"['언어', '성능', '고도화']",기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다,"[0.03565007075667381, -0.7653368711471558, 0.3...",2,"[0.1, 0.1, 0.4, 0.4]"


In [ ]:
# 채용공고에 대한 정보가 있는 wanted_final_with_token.csv 채용공고에 할당해주기 위해 변수 분리
ratio_df = df_sentence[["index", "label_ratios"]]

In [ ]:
# index 기준 merge를 위해 중복 index값 제거
ratio_df.drop_duplicates(subset=['index'], inplace=True)
ratio_df

<ipython-input-121-5de856bbb395>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratio_df.drop_duplicates(subset=['index'], inplace=True)


,index,label_ratios
0,0,"[0.1, 0.1, 0.4, 0.4]"
10,1,"[0.6363636363636364, 0.09090909090909091, 0.09..."
21,2,"[0.16666666666666666, 0.16666666666666666, 0.2..."
33,3,"[0.2727272727272727, 0.2727272727272727, 0.181..."
44,4,"[0.18181818181818182, 0.36363636363636365, 0.1..."
...,...,...
16342,1139,"[0.45454545454545453, 0.36363636363636365, 0.1..."
16353,1141,"[0.25, 0.125, 0.25, 0.375]"
16361,1142,"[0.08333333333333333, 0.08333333333333333, 0.2..."
16373,1143,"[0.0, 0.23529411764705882, 0.17647058823529413..."


In [ ]:
Gonggo = pd.read_csv('./data/wanted_final_with_token.csv')
Gonggo = Gonggo.reset_index()

In [ ]:
Gonggo = pd.merge(Gonggo, ratio_df, on = 'index', how= 'inner')

In [ ]:
# 채용공고 추천에 필요한 변수선택
Gonggo = Gonggo[["공고명", "직무", "유사 직무", "업종", "회사명", "주요업무", "자격요건", "우대사항", "label_ratios"]]

In [ ]:
# Gonggo.to_csv('./data/Gonggo.csv', index=False)

In [ ]:
# df_sentence.to_csv('./data/final_sentence_df.csv', index=False)

In [ ]:
# 모델 저장
# model.save_pretrained('./model/AutoModel_final')
# tokenizer.save_pretrained('./model/AutoTokenizer_final')
# torch.save(kmeans, './model/kmeans_final')

In [ ]:
df_sentence = pd.read_csv("./data/final_sentence_df.csv")
df_sentence.head(3)

,index,공고명,직무,유사 직무,업종,회사명,주요업무,자격요건,우대사항,nouns,sentence,vector,kmeans_label
0,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"['언어', '성능', '고도화']",기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다,"[0.03565007075667381, -0.7653368711471558, 0.3...",2
1,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"['LLM', '학습', 'finetuning', 'LLM']",LLM 학습 및 finetuning을 통해 내부적으로 필요한 LLM 개발을 진행합니다,"[0.026964180171489716, -0.3026787340641022, 0....",3
2,0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"데이터 엔지니어,머신러닝 엔지니어","IT, 컨텐츠",플리토,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분 ML 프레...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"['AI', 'ML', '방향성']","AI , ML 개발 방향성에 대한 전략을 수립합니다","[-0.10177028179168701, -0.4882239103317261, 0....",3


# 명사 분리
토픽모델링에 사용할 군집라벨이 있는 명사 분리

In [ ]:
df_noun = df_sentence[["직무", "nouns", "kmeans_label"]]

In [ ]:
df_noun.head(3)

,직무,nouns,kmeans_label
0,머신러닝 엔지니어,"['언어', '성능', '고도화']",2
1,머신러닝 엔지니어,"['LLM', '학습', 'finetuning', 'LLM']",3
2,머신러닝 엔지니어,"['AI', 'ML', '방향성']",3


In [ ]:
# nouns 칼럼 리스트 안에 있는 따옴표 제거
df_noun['nouns'] = df_noun['nouns'].apply(lambda x: eval(x))

# nouns 칼럼 분리
df_noun = df_noun.explode('nouns')
df_noun = df_noun.reset_index(drop=True)
df_noun['nouns'] = df_noun['nouns'].str.strip()
df_noun.head(10)

<ipython-input-18-d12d36e035c9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_noun['nouns'] = df_noun['nouns'].apply(lambda x: eval(x))


,직무,nouns,kmeans_label
0,머신러닝 엔지니어,언어,2
1,머신러닝 엔지니어,성능,2
2,머신러닝 엔지니어,고도화,2
3,머신러닝 엔지니어,LLM,3
4,머신러닝 엔지니어,학습,3
5,머신러닝 엔지니어,finetuning,3
6,머신러닝 엔지니어,LLM,3
7,머신러닝 엔지니어,AI,3
8,머신러닝 엔지니어,ML,3
9,머신러닝 엔지니어,방향성,3


In [ ]:
# 부동소수점 자료형이 있어 토픽모델링 및 워드클라우드 사용을 위해 문자열로 변환
df_noun['nouns'] = df_noun['nouns'].astype(str)

In [ ]:
# 빈도가 월등히 높은 데이터는 군집에 중복적으로 나타남을 확인
df_noun['nouns'].value_counts().head(50)

데이터       4294
분석        1884
모델        1627
등         1460
운영        1227
nan       1184
구축         930
AI         854
기술         782
딥러닝        653
시스템        653
업무         625
처리         624
관리         607
Python     599
연구         594
다양         594
알고리즘       533
Data       496
ML         479
머신러닝       466
파이         466
프로젝트       458
학습         385
SQL        379
통계         376
가          361
개선         359
해결         356
언어         351
수행         346
최적화        340
구현         338
플랫폼        337
AWS        334
협업         328
DB         317
능숙         311
이용         306
논문         303
학위         298
로          298
클라우드       282
커뮤니케이션     253
관심         252
석사         246
시각화        229
비즈니스       229
성능         228
솔루션        225
Name: nouns, dtype: int64

In [ ]:
exclude_words = ['nan']

df_noun = df_noun[~df_noun['nouns'].apply(lambda x: any(word in x for word in exclude_words))]
df_noun = df_noun[~((df_noun['kmeans_label'] == 2) & (df_noun['nouns'].str.contains('ML')))]
df_noun = df_noun.reset_index(drop=True)
df_noun['nouns'].value_counts()

데이터    4294
분석     1884
모델     1627
등      1460
운영     1227
       ... 
신         1
분석가       1
토         1
위         1
능         1
Name: nouns, Length: 760, dtype: int64

In [ ]:
# df_noun.to_csv('./data/final_noun_df.csv', index=False)